In [3]:
import patsy as ps
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import patsy as ps

In [4]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST.csv')

In [5]:
df_train.isnull().sum()

LID                0
MRP                0
CHANNEL            0
SELLER             0
ORATE              0
CRATE              0
OUPB               0
OLTERM             0
ODATE              0
FPD                0
LA                 0
TMTLM              0
RMTM               0
MDATE              0
LOTV               1
CLTV          163905
NB               198
DTI             6998
BCSAO           3981
CBCSAO         82221
FTHBI            218
LP                 0
PTYPE              0
NUNITS             0
OCCSTAT            0
STATE              0
MSA                0
ZIP                0
MIP           110261
CLDS               0
MIT           110261
RMI                0
FORECLOSED         0
NMONTHS            0
dtype: int64

since there are two many missing info for features: CLTV, MIP, MIT both in training and testing set, I decide to drop those three features.

In [6]:
df_train = df_train.drop(['CLTV','MIP','MIT'],axis = 1)
df_test = df_test.drop(['CLTV','MIP','MIT'],axis = 1)
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,PTYPE,NUNITS,OCCSTAT,STATE,MSA,ZIP,CLDS,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,3/1/2000,C,Suntrust Mortgage Inc.,8.375,8.375,58000,360,7/1/1999,9/1/1999,...,SF,1,P,OK,36420,730,0,N,False,12
1,9.288980e+11,2/1/2000,C,Amtrust Bank,7.500,7.500,204000,180,12/1/1999,2/1/2000,...,SF,1,P,WA,48300,988,0,N,False,21
2,2.229740e+11,1/1/2000,C,"Jpmorgan Chase Bank, Na",7.750,7.750,181000,360,12/1/1999,2/1/2000,...,SF,1,P,VA,40060,238,0,N,False,54
3,5.939500e+11,2/1/2000,B,"Bank Of America, N.A.",8.625,8.625,135000,360,1/1/2000,3/1/2000,...,PU,1,P,AZ,38060,852,0,N,False,16
4,6.773390e+11,2/1/2000,C,Other,7.750,7.750,189000,360,12/1/1999,2/1/2000,...,SF,1,P,IN,29140,479,0,N,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,2/1/2000,C,Amtrust Bank,7.875,7.875,106000,180,11/1/1999,1/1/2000,...,SF,1,I,MI,11460,481,0,N,False,24
164571,6.558220e+11,3/1/2000,R,"Jpmorgan Chase Bank, Na",8.000,8.000,144000,180,2/1/2000,4/1/2000,...,SF,1,P,CA,41740,920,0,N,False,19
164572,6.577330e+11,3/1/2000,R,First Tennessee Bank National Association,8.250,8.250,88000,360,2/1/2000,3/1/2000,...,SF,1,P,WV,26580,255,0,N,False,12
164573,3.145290e+11,1/1/2000,R,"Ge Mortgage Services, Llc",7.875,7.875,99000,240,10/1/1999,12/1/1999,...,SF,1,P,NY,35620,115,0,N,False,217


In [7]:
df_train.isnull().sum()

LID               0
MRP               0
CHANNEL           0
SELLER            0
ORATE             0
CRATE             0
OUPB              0
OLTERM            0
ODATE             0
FPD               0
LA                0
TMTLM             0
RMTM              0
MDATE             0
LOTV              1
NB              198
DTI            6998
BCSAO          3981
CBCSAO        82221
FTHBI           218
LP                0
PTYPE             0
NUNITS            0
OCCSTAT           0
STATE             0
MSA               0
ZIP               0
CLDS              0
RMI               0
FORECLOSED        0
NMONTHS           0
dtype: int64

Now, there are several feartures left with missing values: LOTV, NB, DTI, BCSAO, CBCSAO, FTHBI, I decide to deal with them individually, \
because the fit for them using same package might not be so accuracte for every features. And I found the testing set have similar \
situation with training set, thus, same operation should be processed with testing set also.

Feature: LOTV

In [8]:
df_train.groupby(['LOTV'])['LOTV'].count()

LOTV
4.0         4
5.0         5
6.0         5
7.0         8
8.0        13
        ...  
93.0     1034
94.0     1095
95.0    21377
96.0       61
97.0     5112
Name: LOTV, Length: 94, dtype: int64

Since for this column, there is only one missing value, and the data type is integer, so I don not need to do transformation for this column, \
I can instead fit the missing line using interpolate() function.

In [9]:
df_train['LOTV'].interpolate(method='linear', direction = 'forward', inplace=True)
df_train['LOTV'].isnull().sum()

0

Feature: NB

In [10]:
df_train.groupby(['NB'])['NB'].count()

NB
1.0     65784
2.0     96484
3.0      1760
4.0       322
5.0        15
6.0         6
7.0         2
8.0         3
10.0        1
Name: NB, dtype: int64

For this column, the majority of data is 1.0 or 2.0, so I decide to use fillna method to fit the missing with median(mean is also feasible).

In [11]:
df_train['NB']= df_train['NB'].fillna(df_train['NB'].median())
df_train['NB'].isnull().sum()

0

Feature: DTI BCSAO CBCSAO FTHBI

In [12]:
df_train.groupby(['DTI'])['DTI'].count()

DTI
1.0      31
2.0      47
3.0      96
4.0     134
5.0     139
       ... 
60.0    853
61.0    799
62.0    774
63.0    656
64.0    646
Name: DTI, Length: 64, dtype: int64

In [13]:
df_train.groupby(['BCSAO'])['BCSAO'].count()

BCSAO
350.0    2
354.0    2
372.0    1
375.0    1
400.0    7
        ..
834.0    1
842.0    1
844.0    1
845.0    1
850.0    1
Name: BCSAO, Length: 369, dtype: int64

In [14]:
df_train.groupby(['CBCSAO'])['CBCSAO'].count()

CBCSAO
396.0    1
400.0    4
446.0    1
455.0    1
459.0    1
        ..
824.0    4
826.0    2
828.0    1
832.0    1
834.0    1
Name: CBCSAO, Length: 340, dtype: int64

In [15]:
df_train.groupby(['FTHBI'])['FTHBI'].count()

FTHBI
N    137174
Y     27183
Name: FTHBI, dtype: int64

It looks the column FTHBI is not numerical values, we can make it to numerical by changing to 0 and 1 for this binary situation

In [16]:
df_train["FTHBI"] = df_train["FTHBI"].apply(lambda FTHBI: 0 if FTHBI == 'N' else (1 if FTHBI == 'Y' else None))

By now, we have those four features with numerical values, and we can apply Multivariate Imputation by Chained Equations (MICE) \
to solve for missing values

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Subset numerical features
df_imput = df_train[['DTI', 'BCSAO', 'CBCSAO', 'FTHBI']].copy()
imp = IterativeImputer()
imp.fit(df_imput)
imputed_df = imp.transform(df_imput)
imputed_df = pd.DataFrame(imputed_df, columns=df_imput.columns)

In [18]:
imputed_df

,DTI,BCSAO,CBCSAO,FTHBI
0,29.000000,728.0,701.000000,0.0
1,34.000000,720.0,723.808581,0.0
2,35.000000,648.0,673.912800,0.0
3,46.000000,676.0,671.000000,1.0
4,28.000000,760.0,759.000000,0.0
...,...,...,...,...
164570,37.000000,700.0,689.000000,0.0
164571,37.000000,631.0,661.891634,0.0
164572,46.000000,783.0,737.000000,0.0
164573,32.056782,791.0,791.000000,0.0


Now we have finished with missing values, we need go to deal with categorical features and timefeatures etc.

In [19]:
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,PTYPE,NUNITS,OCCSTAT,STATE,MSA,ZIP,CLDS,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,3/1/2000,C,Suntrust Mortgage Inc.,8.375,8.375,58000,360,7/1/1999,9/1/1999,...,SF,1,P,OK,36420,730,0,N,False,12
1,9.288980e+11,2/1/2000,C,Amtrust Bank,7.500,7.500,204000,180,12/1/1999,2/1/2000,...,SF,1,P,WA,48300,988,0,N,False,21
2,2.229740e+11,1/1/2000,C,"Jpmorgan Chase Bank, Na",7.750,7.750,181000,360,12/1/1999,2/1/2000,...,SF,1,P,VA,40060,238,0,N,False,54
3,5.939500e+11,2/1/2000,B,"Bank Of America, N.A.",8.625,8.625,135000,360,1/1/2000,3/1/2000,...,PU,1,P,AZ,38060,852,0,N,False,16
4,6.773390e+11,2/1/2000,C,Other,7.750,7.750,189000,360,12/1/1999,2/1/2000,...,SF,1,P,IN,29140,479,0,N,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,2/1/2000,C,Amtrust Bank,7.875,7.875,106000,180,11/1/1999,1/1/2000,...,SF,1,I,MI,11460,481,0,N,False,24
164571,6.558220e+11,3/1/2000,R,"Jpmorgan Chase Bank, Na",8.000,8.000,144000,180,2/1/2000,4/1/2000,...,SF,1,P,CA,41740,920,0,N,False,19
164572,6.577330e+11,3/1/2000,R,First Tennessee Bank National Association,8.250,8.250,88000,360,2/1/2000,3/1/2000,...,SF,1,P,WV,26580,255,0,N,False,12
164573,3.145290e+11,1/1/2000,R,"Ge Mortgage Services, Llc",7.875,7.875,99000,240,10/1/1999,12/1/1999,...,SF,1,P,NY,35620,115,0,N,False,217


In [20]:
df_train = pd.concat([df_train.iloc[:,0:16],imputed_df,df_train.iloc[:,20:]], axis=1)

In [21]:
df_train.isnull().sum()

LID           0
MRP           0
CHANNEL       0
SELLER        0
ORATE         0
CRATE         0
OUPB          0
OLTERM        0
ODATE         0
FPD           0
LA            0
TMTLM         0
RMTM          0
MDATE         0
LOTV          0
NB            0
DTI           0
BCSAO         0
CBCSAO        0
FTHBI         0
LP            0
PTYPE         0
NUNITS        0
OCCSTAT       0
STATE         0
MSA           0
ZIP           0
CLDS          0
RMI           0
FORECLOSED    0
NMONTHS       0
dtype: int64

there are some columns with binary values, lets start with the easiest first, RMI is the only one we need to fix

In [22]:
df_train["RMI"] = df_train["RMI"].apply(lambda RMI: 0 if RMI == 'N' else 1)
df_train.groupby(['RMI'])['RMI'].count()

RMI
0    162948
1      1627
Name: RMI, dtype: int64

In [23]:
df_train.groupby(['STATE'])['STATE'].count()

STATE
AK      222
AL     2233
AR     1280
AZ     4675
CA    16776
CO     5041
CT     1871
DC      319
DE      558
FL    13330
GA     6546
HI      766
IA     1364
ID      813
IL     6825
IN     3325
KS     1206
KY     1520
LA     2569
MA     4642
MD     2854
ME      621
MI     8141
MN     3057
MO     3370
MS     1394
MT      507
NC     4186
ND      163
NE     1104
NH     1036
NJ     4770
NM      986
NV     1858
NY     7244
OH     5955
OK     1700
OR     2829
PA     5349
PR      512
RI      770
SC     2250
SD      347
TN     3224
TX    10372
UT     2094
VA     3615
VI       16
VT      301
WA     4414
WI     2888
WV      479
WY      288
Name: STATE, dtype: int64

In [24]:
encodings = df_train.groupby(['SELLER'])["NMONTHS"].count().to_dict()
df_train['SELLER'] = df_train['SELLER'].map(encodings)
encodings = df_train.groupby(['STATE'])['NMONTHS'].mean().to_dict()
df_train['STATE'] = df_train['STATE'].map(encodings)
encodings = df_train.groupby(['ODATE'])['NMONTHS'].mean().to_dict()
df_train['ODATE'] = df_train['ODATE'].map(encodings)
encodings = df_train.groupby(['FPD'])['NMONTHS'].mean().to_dict()
df_train['FPD'] = df_train['FPD'].map(encodings)
#df_train = df_train.merge(encodings, how='left', on='SELLER',indicator = 'Y')
#data.drop('S', axis=1, inplace=True)

In [25]:
categorical_cols = ['MRP','CHANNEL','LP','PTYPE','OCCSTAT',"CLDS"]

In [26]:
for col in df_train.columns:
    if col in categorical_cols:
        a={}
        for i in range(len(df_train[col].unique())):
            a[df_train[col].unique()[i]]=i
        work=df_train[col].map(a)
        df_train[col]=work

In [27]:
df_train

,LID,MRP,CHANNEL,SELLER,ORATE,CRATE,OUPB,OLTERM,ODATE,FPD,...,PTYPE,NUNITS,OCCSTAT,STATE,MSA,ZIP,CLDS,RMI,FORECLOSED,NMONTHS
0,8.118470e+11,0,0,7487,8.375,8.375,58000,360,46.714044,47.611684,...,0,1,0,43.719412,36420,730,0,0,False,12
1,9.288980e+11,1,0,5405,7.500,7.500,204000,180,38.257179,38.251967,...,0,1,0,34.303353,48300,988,0,0,False,21
2,2.229740e+11,2,0,30348,7.750,7.750,181000,360,38.257179,38.251967,...,0,1,0,36.424896,40060,238,0,0,False,54
3,5.939500e+11,1,1,15806,8.625,8.625,135000,360,34.881663,34.870578,...,1,1,0,32.958503,38060,852,0,0,False,16
4,6.773390e+11,1,0,41732,7.750,7.750,189000,360,38.257179,38.251967,...,0,1,0,37.634887,29140,479,0,0,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164570,8.194500e+11,1,0,5405,7.875,7.875,106000,180,37.699625,37.645149,...,0,1,1,29.937108,11460,481,0,0,False,24
164571,6.558220e+11,0,2,30348,8.000,8.000,144000,180,31.725965,31.625283,...,0,1,0,29.789163,41740,920,0,0,False,19
164572,6.577330e+11,0,2,5390,8.250,8.250,88000,360,31.725965,34.870578,...,0,1,0,48.607516,26580,255,0,0,False,12
164573,3.145290e+11,2,2,337,7.875,7.875,99000,240,39.604398,39.444197,...,0,1,0,44.440088,35620,115,0,0,False,217


In [28]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

In [30]:
sm = SMOTE(random_state=42)
df_train = sm.fit_resample(df_train)

TypeError: fit_resample() missing 1 required positional argument: 'y'